<a href="https://colab.research.google.com/github/vamshishashikrishna/LearnAnalytics/blob/main/Bike_Lanes_Asphalt_vs_Cobblestone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
andandand_roads_of_berlin_path = kagglehub.dataset_download('andandand/roads-of-berlin')

print('Data source import complete.')


100%|██████████| 175M/175M [00:02<00:00, 61.1MB/s]

Extracting files...


Data source import complete.


NameError: name 'view' is not defined

## Fine-Tuning a Convolutional Network to Classify Roads as Asphalt or Cobblestone

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
import torch.nn.functional as F


class RoadDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.classes = ['asphalt', 'cobblestone']
        self.images = []
        self.labels = []

        for i, class_name in enumerate(self.classes):
            class_path = self.root_dir / class_name
            for img_path in class_path.glob('*.jpg'):
                self.images.append(img_path)
                self.labels.append(i)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = RoadDataset('/kaggle/input/roads-of-berlin/test', transform=transform)
test_full_dataset = RoadDataset('/kaggle/input/roads-of-berlin/test', transform=transform)

# Split test set into validation and test
test_size = len(test_full_dataset)
val_size = test_size // 2
test_size = test_size - val_size

val_dataset, test_dataset = random_split(test_full_dataset, [val_size, test_size])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

model = models.efficientnet_v2_s(weights='IMAGENET1K_V1')
model.classifier = nn.Linear(model.classifier[1].in_features, 2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

def train_model():
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        epoch_train_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item()

        avg_train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation phase
        model.eval()
        epoch_val_loss = 0.0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                epoch_val_loss += loss.item()

        avg_val_loss = epoch_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        print(f'Epoch [{epoch+1}/{num_epochs}] Train Loss: {avg_train_loss:.4f} Val Loss: {avg_val_loss:.4f}')

    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
train_model()

In [ ]:
input_image = Image.open('/kaggle/input/roads-of-berlin/test/cobblestone/cobblestones_026.jpg')
input_image

In [ ]:
input_tensor = transform(input_image).unsqueeze(0)

In [ ]:
model.eval()
with torch.no_grad():
    output = model(input_tensor.to(device))

In [ ]:
labels = ['asphalt', 'cobblestone']
probs = F.softmax(output, dim=1).cpu().numpy()[0]
list(zip(labels, probs))

## Tasks


---

### **Beginner Level**

1. **Understand the Code**:
    - Run the code in Google Colab, what needs to be changed?
    - Can you run the training loop using the GPU? How do you set it up?
    - Run the code on another image from the test set
    - Identify what each library and function does (e.g., `transforms`, `models`, `random_split`).
    - Explore the distribution of labels in asphalt/cobblestone. Which class is dominant? How does this influence the classifier?
    - Explain the purpose of `Image.open` and why the image needs to be transformed.
3. **Modify Preprocessing**:
    - Use `transforms.RandomHorizontalFlip()` to augment the data.
    - Visualize augmented images


---

### **Intermediate Level**

1. **Experiment with Hyperparameters**:
    
    - Modify the learning rate, batch size, or number of epochs in the training function and observe how the results change.
    - Use a different optimizer (e.g., Adam or RMSprop) instead of the default one.
2. **Visualize Model Performance**:
    - Create a confusion matrix or plot f1 score/loss curves during training.
3. **Checkpoint the Model**:
   - Save the model with best performance on the validation set according to loss or f1 score.
  


---

### **Advanced Level**

1. **Transfer Learning**:
    
    - Replace the model architecture with a different pre-trained model from `torchvision.models` (e.g., ResNet, MobileNet).
    - Change the architecture of the layers that we have added to perform classification

2. **Model Interpretation**:
    
    - Use CAM or Grad-CAM to visualize which parts of the image the model focuses on when making predictions.
  
3. **Model Tracking**:
   - Use [Weights and Biases](https://wandb.ai/) to track the performance of your best models.


---

